In [4]:

import torchvision.transforms as transforms


# Define transforms once outside the function
pytorch_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


def apply_transforms(batch):
    batch["img"] = [pytorch_transforms(img) for img in batch["img"]]
    return batch


import random
from torch.utils.data import Subset
from torchvision import datasets, transforms


In [22]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset

# Define the dataset root directory
root = '/opt/codes/Fed-SCRUB/data'

# Define transformations (optional)
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load training and test datasets
train_dataset = datasets.CIFAR10(root=root, train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root=root, train=False, download=True, transform=transform)

# Combine both datasets
combined_dataset = ConcatDataset([train_dataset, test_dataset])

# Check the total length of the combined dataset
print(f"Total samples: {len(combined_dataset)}")

Total samples: 60000


In [ ]:
from torch.utils.data import ConcatDataset

In [23]:

def configure_balanced_partition(root: str, dataset_name: str, partition_id: int, num_partitions: int, seed: int,
                                 shuffle: bool):
    """
    Load a dataset and partition it so that each partition gets the same number of samples per label.

    Args:
        root (str): Path to dataset storage.
        dataset_name (str): Name of the dataset (currently only "CIFAR10" is supported).
        partition_id (int): The partition index (0 to num_partitions - 1).
        num_partitions (int): Total number of partitions.
        seed (int): Random seed for reproducibility.
        shuffle (bool): Whether to shuffle indices within each label partition.

    Returns:
        Subset: A PyTorch Subset containing the balanced partition of data.
    """
    # Load dataset
    if dataset_name.lower() == "cifar10":
        # Load both training and testing datasets with the same transform

        # Load training and test datasets
        train_dataset = datasets.CIFAR10(root=root, train=True, download=True, transform=transform)
        test_dataset = datasets.CIFAR10(root=root, train=False, download=True, transform=transform)

# Combine both datasets
    else:
        raise ValueError("Unsupported dataset")
    print(len(train_dataset))

    # Validate partition_id
    if not (0 <= partition_id < num_partitions):
        raise ValueError(f"partition_id must be between 0 and {num_partitions - 1}")

    # Group indices by label (CIFAR10 labels are in dataset.targets)
    label_to_indices = {}
    for idx, label in enumerate(test_dataset.targets):
        label_to_indices.setdefault(label, []).append(idx)

    partition_indices = []

    # For each label, shuffle and evenly split indices
    for label, indices in label_to_indices.items():
        # Set the seed for reproducibility, then shuffle the indices for this label.
        random.seed(seed)
        random.shuffle(indices)

        # Determine the size of each partition for this label.
        base_size = len(indices) // num_partitions
        extra = len(indices) % num_partitions  # Some partitions may get one extra sample

        # Compute starting index for the current partition.
        # Partitions with an index less than `extra` receive one extra sample.
        start_idx = sum(base_size + 1 if i < extra else base_size for i in range(partition_id))
        part_size = base_size + 1 if partition_id < extra else base_size
        end_idx = start_idx + part_size

        label_partition = indices[start_idx:end_idx]

        # Optionally shuffle the slice from this label if desired.
        if shuffle:
            random.shuffle(label_partition)

        partition_indices.extend(label_partition)

    # Optionally shuffle the combined partition indices.
    if shuffle:
        random.shuffle(partition_indices)

    return partition_indices

In [24]:
incides = []
for i in [0,1,2,3]:
    incides.append(configure_balanced_partition("/opt/codes/Fed-SCRUB/data", "cifar10", i, 4, 42, True))

60000


AttributeError: 'ConcatDataset' object has no attribute 'targets'

In [17]:
whole = set()
for i in incides:
    print(len(i))
    whole = whole.union(set(i))

print(len(whole))


2500
2500
2500
2500
10000
